This is a simple NN approach.
Training and validation loss seem to converge very quickly.
If you have any suggestions, I'd be happy to hear them!

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import regularizers

In [ ]:
# Import data
train = pd.read_csv('../input/train.csv')
unknown = pd.read_csv('../input/test.csv')

In [ ]:
# Scale data
y_nn = train.target.reset_index(drop=True)
X_nn = train.drop(['ID_code', 'target'], axis = 1)
X_unknown = unknown.drop(['ID_code'], axis=1)
scaler = StandardScaler()
X_nn = scaler.fit_transform(X_nn)
X_unknown = scaler.transform(X_unknown)
# train test split
X_train, X_test, y_train, y_test = train_test_split(X_nn,y_nn,test_size=0.25)
y_test=y_test.values
y_train=y_train.values

In [ ]:
# Callback and optimizer
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', 
                                  patience=5, verbose=1,min_delta=1e-5,restore_best_weights=True)
]
optimizer=keras.optimizers.Adam(lr=0.0005)

In [ ]:
# make model
model=Sequential()
model.add(Dense(512, activation='tanh',
         kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='tanh',
         kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
auc_list=[]

In [ ]:
# fit model to training data
model.fit(X_train, y_train, 
          epochs=75, batch_size=512,
          callbacks=callbacks,
          validation_data = (X_test, y_test), verbose=1)
print('Model fitted. Predicting & Evaluating...')
nn_pred = model.predict(X_test)
auc_score=roc_auc_score(y_test, nn_pred)
nn_pred_train=model.predict(X_train)
auc_score_train=roc_auc_score(y_train, nn_pred_train)
auc_list.append(auc_score)
print('Test score: {}\nTraining Score:{}'.format(auc_score,auc_score_train))
print(auc_list)

In [ ]:
# plot training progress
nn_prev_loss_history=model.history.history['loss']
nn_prev_valloss_history=model.history.history['val_loss']
plt.plot(nn_prev_loss_history)
plt.plot(nn_prev_valloss_history)
plt.suptitle('Training')
plt.ylabel('Binary crossentropy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# fit model to all known data
model.fit(X_nn, y_nn, 
          epochs=6, batch_size=512,
          callbacks=callbacks,
          validation_split=0.002, shuffle=True, verbose=1)
print('Model fitted. Predicting & Evaluating...')
nn_pred = model.predict(X_test)
auc_score=roc_auc_score(y_test, nn_pred)
nn_pred_train=model.predict(X_train)
auc_score_train=roc_auc_score(y_train, nn_pred_train)
print('Test score: {}\nTraining Score:{}'.format(auc_score,auc_score_train))
preds = model.predict(X_unknown)

In [ ]:
# make submission
submission = pd.DataFrame({"ID_code" : unknown['ID_code'].values,
                           "target" : preds[:,0]})
submission.to_csv('submission.csv', index = False, header = True)
display(submission.head(15))
display(submission.tail(15))